# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
import random

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bilibino,68.0546,166.4372,5.41,4.24,35,1.64,RU,1688644249
1,1,bethel,41.3712,-73.4140,26.34,24.43,0,0.00,US,1688644198
2,2,adamstown,-25.0660,-130.1015,20.52,20.64,70,4.90,PN,1688644251
3,3,port mathurin,-19.6833,63.4167,24.04,24.33,19,8.46,MU,1688644253
4,4,mana,5.6592,-53.7767,24.46,25.36,100,3.77,GF,1688644254


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
baseurl = "https://api.geoapify.com/v1/forecast/weather"
params = {"apiKey": geoapify_key}


# Configure the map plot
city_data_df["colour"] = ""

for index, row in city_data_df.iterrows():
    city_data_df.at[index,"colour"] = f"#{random.randint(0,0xFFFFFF):06x}"

# Display the map
city_data_df.hvplot.points(x="Lng", y="Lat", 
                           size="Humidity", geo=True, tiles="CartoLight",color="colour",alpha = 0.5)

c:\Users\gello\anaconda3\envs\dev\lib\site-packages\holoviews\util\transform.py:674: RuntimeWarning: invalid value encountered in sqrt
  data = fn(*args, **kwargs)


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (colour,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
#Criteria: max temp: 21-27, wind speed: < 4.5, cloudiness == 0.
filtered_df = city_data_df[
    (city_data_df["Max Temp"] >= 21) & (city_data_df["Max Temp"] <= 27) &
    (city_data_df["Wind Speed"] < 4.5) &
    (city_data_df["Cloudiness"] == 0)
# Drop any rows with null values
].dropna()

# Display sample data
filtered_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,colour
1,1,bethel,41.3712,-73.4140,26.34,24.43,0,0.00,US,1688644198,#88848c
49,49,nantucket,41.2835,-70.0995,23.03,21.95,0,3.12,US,1688644312,#a889a4
52,52,holualoa,19.6228,-155.9522,21.91,21.69,0,4.12,US,1688644316,#95fca4
111,111,bois-le-roi,48.4735,2.7046,25.39,23.45,0,1.54,FR,1688644397,#5dde66
122,122,jamestown,42.0970,-79.2353,24.29,19.38,0,2.06,US,1688644174,#c5d854


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
1,bethel,US,41.3712,-73.4140,24.43,
49,nantucket,US,41.2835,-70.0995,21.95,
52,holualoa,US,19.6228,-155.9522,21.69,
111,bois-le-roi,FR,48.4735,2.7046,23.45,
122,jamestown,US,42.0970,-79.2353,19.38,
136,palmer,US,42.1584,-72.3287,23.52,
166,selma,US,36.5708,-119.6121,18.33,
209,lancaster,US,40.2504,-76.2499,25.01,
223,kailua-kona,US,19.6406,-155.9956,23.98,
231,moab,US,38.5733,-109.5498,22.64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "limit":4,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    #params["filter"] = f"circle:{lat},{lon},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    params["filter"] = f"circle:{lon},{lat},{radius}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df["colour"] = ""
for index, row in hotel_df.iterrows():
    hotel_df.at[index,"colour"] = f"#{random.randint(0,0xFFFFFF):06x}"
hotel_df


Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
nantucket - nearest hotel: No hotel found
holualoa - nearest hotel: Kona Hotel
bois-le-roi - nearest hotel: Relais du Manoir
jamestown - nearest hotel: DoubleTree Jamestown
palmer - nearest hotel: No hotel found
selma - nearest hotel: Best Western
lancaster - nearest hotel: Mountain Trail Motel
kailua-kona - nearest hotel: Kona Seaside Hotel
moab - nearest hotel: Kokopelli Inn
kolwezi - nearest hotel: Guest House le Manguier
montmagny - nearest hotel: Entreprises val de france
laurel - nearest hotel: Quality Inn & Suites
les sables-d'olonne - nearest hotel: L'Atlantic
colonia - nearest hotel: Wasserturm Hotel Cologne
lubumbashi - nearest hotel: Hôtel de Ville de Lubumbashi
fort irwin - nearest hotel: Landmark Inn
tillmans corner - nearest hotel: Red Roof Inn


,City,Country,Lat,Lng,Humidity,Hotel Name,colour
1,bethel,US,41.3712,-73.4140,24.43,Hampton Inn Danbury,#f8bc6e
49,nantucket,US,41.2835,-70.0995,21.95,No hotel found,#63af51
52,holualoa,US,19.6228,-155.9522,21.69,Kona Hotel,#020ae9
111,bois-le-roi,FR,48.4735,2.7046,23.45,Relais du Manoir,#85c63c
122,jamestown,US,42.0970,-79.2353,19.38,DoubleTree Jamestown,#db7e0c
136,palmer,US,42.1584,-72.3287,23.52,No hotel found,#099783
166,selma,US,36.5708,-119.6121,18.33,Best Western,#e16dd0
209,lancaster,US,40.2504,-76.2499,25.01,Mountain Trail Motel,#bc5dfe
223,kailua-kona,US,19.6406,-155.9956,23.98,Kona Seaside Hotel,#7c53c2
231,moab,US,38.5733,-109.5498,22.64,Kokopelli Inn,#168ad4


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
# Configure the map plot
hover_tooltip = [("Hotel","Hotel Name"),("Country","Country")]

# Display the map
hotel_df.hvplot.points(x="Lng", y="Lat", 
                           size=85, geo=True, tiles="CartoLight",color="colour",alpha = 0.5,hover_cols=["Hotel Name","Country"])

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (colour,Hotel Name,Country)